1. Visual Studio 설치(C++ 패키치 포함)
2. pytorch 설치
3. pip install mxnet(numpy version < 1.21.0 필수!!!)
4. pip install gluonnlp 
5. pip install sentencepiece

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [7]:
import gluonnlp as nlp

In [8]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [9]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [10]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [11]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [12]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [13]:
transform = nlp.data.BERTSentenceTransform(
            tok, max_seq_length=max_len, pad=True, pair=False)

In [15]:
sample = "오늘은 날씨가 좋습니다"
print(tok(sample))
sentences = transform([sample])
print(sentences)

['▁오늘', '은', '▁날씨', '가', '▁좋', '습니다']
(array([   2, 3419, 7086, 1408, 5330, 4204, 6701,    3,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1]), array(8), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
